#### Краткое введение в декораторы
Декораторы функций дают возможность «помечать» функции в исходном коде, тем или иным способом дополняя их поведение.
Декоратор – это вызываемый объект, который принимает другую функцию в качестве аргумента (декорируемую функцию). Декоратор может производить какието операции с функцией и возвращает либо ее саму, либо другую заменяющую ее функцию или вызываемый объект

#### Пример 7.1. 
Декоратор обычно заменяет одну функцию другой

In [4]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

# Иначе говоря, в предположении, что декоратор с именем deco,следующий код:
@deco
def target():
    print('running target()')

target()

running inner()


In [5]:
target

<function __main__.deco.<locals>.inner()>

эквивалентен такому:

In [6]:
def target():
    print('running target()')
    target = deco(target)

target()

running target()


UnboundLocalError: local variable 'target' referenced before assignment

Конечный результат одинаков: в конце обоих фрагментов имя target необязательно ссылается на исходную функцию target, это может быть любая другая функция, возвращенная в результате вызова deco(target).

#### Когда Python выполняет декораторы

Главное свойство декораторов – то, что они выполняются сразу после определения декорируемой функции. Обычно на этапе импорта (т. е. когда Python загружает модуль).

#### Пример 7.2. 
Модуль registration.py

In [ ]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x7f71ec1d1000>)
running register(<function f2 at 0x7f71ec1d0670>)
running main()
registry -> [<function f1 at 0x7f71ec1d1000>, <function f2 at 0x7f71ec1d0670>]
running f1()
running f2()
running f3()


Основная цель примера 7.2 – подчеркнуть, что декораторы функций выполняются сразу после импорта модуля, но сами декорируемые функции – только в результате явного вызова. В этом проявляется различие между этапом импорта и этапом выполнения в Python.

#### Паттерн Стратегия, дополненный декоратором

#### Пример 7.3. 
Список promos заполняется декоратором promotion

In [ ]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func


@promotion
def fidelity_promo(order):
        """5 procents for customers with at least 1000 loyalty points"""
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


@promotion
def bulk_item_promo(order):
        """10% discount for each LineItem with a minimum order of 20 units"""
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


@promotion
def large_order_promo(order):
        """7% discount for orders including at least 10 different items"""
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0


def best_promo(order):
    """Choose max possible discount"""
    return max(promo(order) for promo in promos)

#### Правила видимости переменных

#### Пример 7.4. 
Функция, читающая локальную и глобальную переменную

In [ ]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [ ]:
b = 6
f1(3)

3
6


#### Пример 7.5. 
Переменная b локальна, потому что ей присваивается значение в теле функции

In [ ]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

Компилируя тело этой функции, Python решает, что b – локальная переменная, т. к. ей присваивается значение внутри функции
Это не ошибка, а осознанный выбор: Python не заставляет нас объявлять переменные, но предполагает, что всякая переменная, которой присваивается значение в теле функции, локальна.

In [ ]:
def f3(a):
    global b
    print(a)
    print(b)
    b = 9
f3(3)

3
9


In [ ]:
b

9

In [ ]:
f3(3)

3
9


#### Пример 7.6. 
Дизассемблированная функция f1 из примера 7.4

In [ ]:
from dis import dis
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


#### Пример 7.7. 
Дизассемблированная функция f2 из примера 7.5

In [ ]:
from dis import dis
dis(f2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


#### Замыкания

Замыкание – это функция с расширенной областью видимости, которая охватывает все неглобальные переменные, на которые есть ссылки в теле функции, хотя они в нем не определены

#### Пример 7.8. 
average_oo.py: класс для вычисления накопительного среднего

In [ ]:
class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [ ]:
avg = Averager()
avg(10)

10.0

In [ ]:
avg(11)

10.5

In [ ]:
avg(12)

11.0

#### Пример 7.9. 
average.py: функция высшего порядка для вычисления накопительного
среднего

In [ ]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

avg = make_averager()
avg(10)

10.0

In [ ]:
avg(11)

10.5

In [ ]:
avg(12)

11.0

Обратите внимание, что series – локальная переменная make_averager, потому что инициализация series = [] производится в теле этой функции. Но к моменту вызова avg(10) функция make_averager уже вернула управление, и ее локальная область видимости уничтожена. 
Внутри averager series является свободной переменной. Этот технический термин означает, что переменная не связана в локальной области видимости

In [ ]:
avg.__code__.co_varnames

('new_value', 'total')

In [ ]:
avg.__code__.co_freevars

('series',)

In [ ]:
avg.__closure__

(<cell at 0x7fdd4af75390: list object at 0x7fdd4ac71c80>,)

In [ ]:
avg.__closure__[0].cell_contents

[10, 11, 12]

Замыкание – это функция, которая запоминает привязки свободных переменных, существовавшие на момент определения функции, так что их можно использовать впоследствии при вызове функции, когда область видимости, в которой она была определена, уже не существует

#### Объявление nonlocal

#### Пример 7.13. 
Неправильная функция высшего порядка для вычисления накопительного среднего без хранения всей истории

In [ ]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total/count

    return averager

avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

Проблема в том, что предложение count += 1 означает то же самое, что count = count + 1, где count – число или любой неизменяемый тип. То есть мы по сути дела присваиваем count значение в теле averager, делая ее тем самым локальной переменной. То же относится к переменной total.
В примере 7.9 этой проблемы не было, потому что мы ничего не присваивали переменной series; мы лишь вызывали series.append и передавали ее функциям sum и len. То есть воспользовались тем, что список – изменяемый тип.
Однако переменные неизменяемых типов – числа, строки, кортежи и т. д. – разрешается только читать, но не изменять. Если попытаться перепривязать такую переменную, как в случае count = count + 1, то мы неявно создадим локальную переменную count. Она уже не является свободной и потому не запоминается в
замыкании.

In [ ]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count

    return averager

avg = make_averager()
avg(10)

10.0

In [ ]:
avg(11)

10.5

In [ ]:
avg(12)

11.0

#### Реализация простого декоратора

#### Пример 7.15. 
Простой декоратор для вывода времени выполнения функции

In [ ]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '. join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

#### Пример 7.16. 
Использование декоратора clock

In [ ]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print ('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print ('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[2.12446992s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[2.17122686s] factorial(1) -> 1
[44.06751261s] factorial(2) -> 2
[59.35804009s] factorial(3) -> 6
[66.80371918s] factorial(4) -> 24
[97.84870092s] factorial(5) -> 120
[106.06643757s] factorial(6) -> 720
6! = 720


Напомним, что код

In [ ]:
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

на самом деле эквивалентен следующему:

In [ ]:
def factorial(n):
 return 1 if n < 2 else n*factorial(n-1)
 factorial = clock(factorial)

То есть в обоих случаях декоратор clock получает функцию factorial в качестве аргумента func (см. пример 7.15). Затем он создает и возвращает функцию clocked, которую интерпретатор Python за кулисами связывает с именем factorial. На самом деле, если импортировать factorial и вывести атрибут __name__ функции factorial, то мы увидим

In [ ]:
factorial.__name__

'clocked'

Таким образом, factorial действительно хранит ссылку на функцию clocked. Начиная с этого момента, при каждом вызове factorial(n) выполняется clocked(n). А делает clocked вот что:
1. Запоминает начальный момент времени t0.
2. Вызывает исходную функцию factorial и сохраняет результат.
3. Вычисляет, сколько прошло времени.
4. Форматирует и печатает собранные данные.
5. Возвращает результат, сохраненный на шаге 2.
Это типичное поведение декоратора: заменить декорируемую функцию новой, которая принимает те же самые аргументы и (как правило) возвращает то, что должна была бы вернуть деко

Декоратор clock, реализованный в примере 7.15, имеет ряд недостатков: он не поддерживает именованные аргументы и маскирует атрибуты __name__ и __doc__ декорированной функции. В примере 7.17 используется декоратор functools.wraps, который копирует необходимые атрибуты из func в clocked. К тому же, в этой новой версии правильно обрабатываются именованные аргументы.

#### Пример 7.17. 
Улучшенный декоратор clock

In [ ]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '. join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted(kwargs.items())]
            arg_lst.append(', '. join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [ ]:
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__ == '__main__':
    print ('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling factorial(6)
[0.00000119s] factorial(1) -> 1
[0.00004697s] factorial(2) -> 2
[0.00006008s] factorial(3) -> 6
[0.00007129s] factorial(4) -> 24
[0.00008273s] factorial(5) -> 120
[0.00009513s] factorial(6) -> 720
6! = 720


In [ ]:
factorial.__name__

'factorial'

### Декораторы в стандартной библиотеке

#### Кэширование с помощью functools.lru_cache

Декоратор functools.lru_cache очень полезен на практике. Он реализует «запоминание» (memoization): прием оптимизации, смысл которого заключается в сохранении результатов предыдущих дорогостоящих вызовов функции, что позволяет избежать повторного вычисления с теми же аргументами, что и раньше.
Акроним LRU расшифровывается как «Least Recently Used» (последний использованный); это означает, что рост кэша ограничивается путем вытеснения тех элементов, к которым давно не было обращений.

#### Пример 7.18. 
Очень накладный рекурсивный способ вычисления n-ого числа Фибоначчи

In [ ]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci (n - 2) + fibonacci(n-1)

if __name__ == '__main__':
    print(fibonacci(6))

[0.00000072s] fibonacci(0) -> 0
[0.00002933s] fibonacci(1) -> 1
[0.01396132s] fibonacci(2) -> 1
[0.00000072s] fibonacci(1) -> 1
[0.00000095s] fibonacci(0) -> 0
[0.00000048s] fibonacci(1) -> 1
[0.00002122s] fibonacci(2) -> 1
[0.00004244s] fibonacci(3) -> 2
[0.01402807s] fibonacci(4) -> 3
[0.00000048s] fibonacci(1) -> 1
[0.00000048s] fibonacci(0) -> 0
[0.00000048s] fibonacci(1) -> 1
[0.00001836s] fibonacci(2) -> 1
[0.00003695s] fibonacci(3) -> 2
[0.00000048s] fibonacci(0) -> 0
[0.00000048s] fibonacci(1) -> 1
[0.00001931s] fibonacci(2) -> 1
[0.00000048s] fibonacci(1) -> 1
[0.00000072s] fibonacci(0) -> 0
[0.00000048s] fibonacci(1) -> 1
[0.00040817s] fibonacci(2) -> 1
[0.00044966s] fibonacci(3) -> 2
[0.00048876s] fibonacci(4) -> 3
[0.00054431s] fibonacci(5) -> 5
[0.01459312s] fibonacci(6) -> 8
8


#### Пример 7.19. 
Более быстрая реализация с использованием кэширования

In [ ]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci (n - 2) + fibonacci(n-1)

if __name__ == '__main__':
    print(fibonacci(6))

[0.00000072s] fibonacci(0) -> 0
[0.00000095s] fibonacci(1) -> 1
[0.00011706s] fibonacci(2) -> 1
[0.00000095s] fibonacci(3) -> 2
[0.00013804s] fibonacci(4) -> 3
[0.00000048s] fibonacci(5) -> 5
[0.00016856s] fibonacci(6) -> 8
8


#### Одиночная диспетчеризация и обобщенные функции